In [1]:
import syft as sy
sy.requires(">=0.8-beta")

/home/teo/anaconda3/envs/PySyft/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


✅ The installed version of syft==0.8.0b3 matches the requirement >=0.8b0


add_credentials_for_key


In [2]:
node = sy.orchestra.launch(name="test-domain-1", processes=1, reset=True)

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


In [3]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [4]:
from syft.core.node.new.new_policy import CustomOutputPolicy
@sy.serializable()
class RepeatedCallPolicy(CustomOutputPolicy):
    __canonical_name__ = "RepeatedCallPolicy"
    from typing import List, Dict, Any

    n_calls: int
    downloadable_output_args: List[str]
    state: Dict[Any, Any] = {}
    
    __attr_allowlist__ = [
        "n_calls",
        "downloadable_output_args",
    ]

    def __init__(self, n_calls=1, downloadable_output_args=None):
        super().__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args if not None else []
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object):
        output_dict = {}
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict

In [5]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])
print(policy.init_args, policy.init_kwargs)
a_obj = sy.ActionObject.from_obj({'y': [1,2,3]})
policy.apply_output(a_obj)

AttributeError: 'RepeatedCallPolicy' object has no attribute 'kwargs'

In [ ]:
import numpy as np
x = np.array([1,2,3])
x_pointer = sy.ActionObject.from_obj(x)
domain_client.api.services.action.save(x_pointer)

<class 'syft.core.node.new.response.SyftSuccess'>: <class 'syft.core.node.new.numpy.NumpyArrayObject'> saved

In [ ]:
obj = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])
obj

```python
class RepeatedCallPolicy:
  id: str = 76102705841146199c198147f6d2002c
  outputs: str = []
  state_type: str = None
  node_uid: str = None
  init_args: str = ()
  kwargs: str = {'n_calls': 1, 'downloadable_output_args': ['y']}
  n_calls: str = 2
  downloadable_output_args: str = ['y']
  state: str = {'counts': 0}

```

In [ ]:
obj

```python
class RepeatedCallPolicy:
  id: str = 76102705841146199c198147f6d2002c
  outputs: str = []
  state_type: str = None
  node_uid: str = None
  init_args: str = ()
  kwargs: str = {'n_calls': 1, 'downloadable_output_args': ['y']}
  n_calls: str = 2
  downloadable_output_args: str = ['y']
  state: str = {'counts': 0}

```

In [ ]:
obj.kwargs

{'n_calls': 1, 'downloadable_output_args': ['y']}

In [ ]:
from syft.core.node.new.new_policy import ExactMatch
@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}

syft.core.node.new.new_policy.ExactMatch
UserPolicy


TypeError: issubclass() arg 1 must be a class

In [ ]:
domain_client.api.services.code.request_code_execution(func)

what kind of output_policy syft.core.node.new.policy.UserPolicy <class 'syft.core.node.new.policy.UserPolicy'>


```python
class Request:
  id: str = 551ea00c9485496c9aa0552f9a6da07a
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-03-22 06:39:57
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "0fc6534a95478c244239617395e5902014db59db10c25e2089680a32c0db3ac3"
  changes: str = [syft.core.node.new.request.UserCodeStatusChange]

```

In [ ]:
request = domain_client.notifications[-1].link
change = func = request.changes[-1]
change

```python
class UserCodeStatusChange:
  id: str = cb917806c3c4495b9d17864434454a95
  linked_obj: str = <<class 'syft.core.node.new.user_code.UserCode'>: d3483f27ffa24f27891cd2dfe098f921@<Node: 7bca415d13ed1ec841f0d0aede098dbb>
  value: str = UserCodeStatus.EXECUTE
  match_type: str = True

```

In [ ]:
request.approve()

<class 'syft.core.node.new.response.SyftSuccess'>: Request 551ea00c9485496c9aa0552f9a6da07a changes applied

In [ ]:
func = request.changes[-1].link
func

```python
class UserCode:
  id: str = d3483f27ffa24f27891cd2dfe098f921
  user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  raw_code: str = "@sy.syft_function(
    input_policy=sy.ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}
"
  input_policy: str = syft.core.node.new.policy.ExactMatch
  input_policy_state: str = None
  output_policy: str = syft.core.node.new.policy.UserPolicy
  output_policy_state: str = None
  parsed_code: str = "

def user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_7a4aa2531a85db30bc30d3f6e5092cfb68f45feb0c0d0807a32229d527e87829(x):

    def func(x):
        return {'y': (x + 1)}
    result = func(x=x)
    return result
"
  service_func_name: str = "func"
  unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_7a4aa2531a85db30bc30d3f6e5092cfb68f45feb0c0d0807a32229d527e87829"
  user_unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f"
  code_hash: str = "7a4aa2531a85db30bc30d3f6e5092cfb68f45feb0c0d0807a32229d527e87829"
  signature: str = (x) -> Dict[str, Any]
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.EXECUTE: 'execute'>}
  input_kwargs: str = ['x']
  outputs: str = []
  input_policy_init_args: str = None
  output_policy_init_args: str = {'n_calls': 1, 'downloadable_output_args': ['y']}
  enclave_metadata: str = None

```

In [ ]:
result = func.unsafe_function(x=x_pointer)
result

Exception detected in make_action_side_effect 'int' object has no attribute 'syft_lineage_id'


,key,type,id
0,y,"array([2, 3, 4])",75ebb9b2eef945909c9b364dc5079039


In [ ]:
result

,key,type,id
0,y,"array([2, 3, 4])",75ebb9b2eef945909c9b364dc5079039


In [ ]:
final_result = request.accept_by_depositing_result(result) 
final_result

message='Policy Code Submitted'


<class 'syft.core.node.new.response.SyftSuccess'>: Request 551ea00c9485496c9aa0552f9a6da07a changes applied

In [ ]:
res = domain_client.api.services.code.func(x=x_pointer)
res

output_policy_state <class 'bytes'>


{'y': array([2, 3, 4])}

In [ ]:
assert (res["y"] == np.array([2, 3, 4])).all()

In [ ]:
assert set(res.keys()) == set(list("y"))

In [ ]:
domain_client.api.services.code.get_all()

,type,id,status,service_func_name
0,syft.core.node.new.user_code.UserCode,d3483f27ffa24f27891cd2dfe098f921,"{NodeView(node_name='test-domain-1', verify_ke...",func


In [ ]:
domain_client.api.services.policy.get_all()

,type,id
0,syft.core.node.new.policy.UserPolicy,d741f3d32dcd4d1c86aa19383542730a


In [ ]:
output_policy = domain_client.api.services.policy.get_all()[0]

In [ ]:
sy.orchestra.land("test-domain-1")

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]
